In [2]:
# from tensorflow.keras.appliications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import AUC,Precision,Recall
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation, RandomZoom
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,Tensorboard

from tensorflow.keras.mixed_precision import set_global_policy
import numpy as np
import os

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)


2025-05-06 22:01:58.648736: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 22:01:58.665875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746540118.684994   18103 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746540118.689560   18103 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 22:01:58.707200: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
model = models.load_model('../checkpoint/model3_50.keras')
     

I0000 00:00:1746540123.895568   18103 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
model.summary()
# model.get_layer('efficientnetb0').summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 3)    │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         8,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,474,340 (43.77 MB)

 Trainable params: 3,707,885 (14.14 MB)

 Non-trainable params: 350,683 (1.34 MB)

 Optimizer params: 7,415,772 (28.29 MB)

In [28]:
# rebuild model

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
conv_layer = model.get_layer('conv2d')
conv_weights = conv_layer.get_weights()

effnet = model.get_layer('efficientnetb0')
effnet_weights = effnet.get_weights()

input_layer = Input(shape=(224, 224, 1), name = 'grayscale_input')
x = Conv2D(3, (3, 3), padding = 'same', activation = 'relu',name = 'conv2d')(input_layer)

effnet = EfficientNetB0(include_top=False, weights=None, input_tensor=None ,pooling='None')
effnet.set_weights(effnet_weights)

x = effnet(x)
x = GlobalAveragePooling2D()(x)
output = Dense(7,activation='sigmoid', name = 'dense')(x)

new_model = Model(inputs=input_layer, outputs=output)
dense_layer = model.get_layer('dense')
new_model.get_layer('dense').set_weights(dense_layer.get_weights())
new_model.summary()

# new_model.get_layer('efficientnetb0').summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ grayscale_input (InputLayer)    │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 3)    │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         8,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,058,568 (15.48 MB)

 Trainable params: 4,016,545 (15.32 MB)

 Non-trainable params: 42,023 (164.16 KB)

ValueError: No such layer: block6a_expand_activation. Existing layers are: ['grayscale_input', 'conv2d', 'efficientnetb0', 'global_average_pooling2d_4', 'dense'].